In [8]:
# Four-Team Tournament EV Calculator with Database Query Integration

import mysql.connector
from collections import defaultdict

# Connect to the database
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get PotentialPayout for championship bets
query = """
    SELECT 
        legs.ParticipantName,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NFL'
        AND legs.EventType = 'Championship'
    GROUP BY legs.ParticipantName;
"""

cursor.execute(query)

# Store payouts in a dictionary
payouts = defaultdict(lambda: {'championship': 0})
for participant_name, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()
    payouts[normalized_name]['championship'] = float(total_payout)

cursor.close()
conn.close()

# Matchups and probabilities (example values for proof of concept)
team_probabilities = {
    "kansas city chiefs": {"semifinal_prob": 0.51, "championship_probs": {"detroit lions": 0.45, "philadelphia eagles": 0.55}},
    "baltimore ravens": {"semifinal_prob": 0.49, "championship_probs": {"detroit lions": 0.5, "philadelphia eagles": 0.60}},
    "detroit lions": {"semifinal_prob": 0.65, "championship_probs": {"kansas city chiefs": 0.55, "baltimore ravens": 0.5}},
    "philadelphia eagles": {"semifinal_prob": 0.35, "championship_probs": {"kansas city chiefs": 0.45, "baltimore ravens": 0.40}},
}

# Function to calculate portfolio EV for a given semifinal outcome
def calculate_portfolio_ev(winning_team):
    ev = 0

    # Calculate EV contributions for all potential championship matchups
    for opponent, win_prob in team_probabilities[winning_team]["championship_probs"].items():
        opponent_prob = team_probabilities[opponent]["semifinal_prob"]
        ev += (
            win_prob * opponent_prob * payouts[winning_team]["championship"] +
            (1 - win_prob) * opponent_prob * payouts[opponent]["championship"]
        )

    return ev

# Calculate portfolio EVs for each scenario
scenarios = [
    ("kansas city chiefs", "baltimore ravens"),  # Chiefs beat Ravens
    ("baltimore ravens", "kansas city chiefs"),  # Ravens beat Chiefs
    ("detroit lions", "philadelphia eagles"),  # Lions beat Eagles
    ("philadelphia eagles", "detroit lions"),  # Eagles beat Lions
]

ev_results = {}
for winner, _ in scenarios:
    ev = calculate_portfolio_ev(winner)
    ev_results[f"{winner.title()} wins semifinal"] = ev

# Display results
for outcome, ev in ev_results.items():
    print(f"Portfolio EV if {outcome}: ${ev:.2f}")


Portfolio EV if Kansas City Chiefs wins semifinal: $249254.49
Portfolio EV if Baltimore Ravens wins semifinal: $162250.33
Portfolio EV if Detroit Lions wins semifinal: $258334.75
Portfolio EV if Philadelphia Eagles wins semifinal: $110585.32
